# cooking the best possible web search result (in termes of relevance and speed)

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## test lib

In [2]:
from agentic_search.lib import get_llm

llm = get_llm("reasoning", False, "openai")
res = llm.invoke("""Reasoning Puzzle: “Who Took the Missing Book?”

Four friends—Alice, Bob, Charlie, and Diane—are having a heated discussion about a missing book. Each person makes two statements. You know that exactly four of the total eight statements are true, and the other four are false. Based on their statements, figure out who actually took the book and which statements are true.

The Statements
Alice
1A. “I didn’t take the book.”
1B. “Bob took it.”

Bob
2A. “Charlie is lying when he says he didn’t take the book.”
2B. “Diane didn’t take the book.”

Charlie
3A. “I didn’t take the book.”
3B. “One of the two statements made by Diane is true.”

Diane
4A. “Alice is lying about Bob taking the book.”
4B. “Charlie must have taken the book.”

Your Task
Determine which friend actually took the missing book.
For each statement (1A, 1B, 2A, 2B, 3A, 3B, 4A, 4B), decide whether it is true or false, making sure to satisfy the condition that exactly four statements in total are true.""")
res

AIMessage(content='To solve this puzzle, we need to determine which four statements are true and which four are false, while also identifying who took the missing book. Let\'s analyze the statements:\n\n1. **Alice\'s Statements:**\n   - 1A. "I didn’t take the book."\n   - 1B. "Bob took it."\n\n2. **Bob\'s Statements:**\n   - 2A. "Charlie is lying when he says he didn’t take the book."\n   - 2B. "Diane didn’t take the book."\n\n3. **Charlie\'s Statements:**\n   - 3A. "I didn’t take the book."\n   - 3B. "One of the two statements made by Diane is true."\n\n4. **Diane\'s Statements:**\n   - 4A. "Alice is lying about Bob taking the book."\n   - 4B. "Charlie must have taken the book."\n\nLet\'s consider the possibilities:\n\n- If Alice took the book, then 1A is false. This would make 1B false as well, since Bob didn\'t take it. This would mean both of Alice\'s statements are false, which is possible. However, if Alice took the book, then Diane\'s statement 4A would be true, and 4B would be 

## test functions

In [1]:
user_query = "what is the latest top loser in the American stock market"

In [3]:
from agentic_search.functions.web import get_serp_links

links = await get_serp_links(user_query)
links


[DEBUG] 
serp results for query what is the latest top loser in the American stock market: [{'title': 'Top Losers Today - Largest Stock Losses in the US Stock Market - Finsuba', 'href': 'https://www.finsuba.com/leaderboards/us-stocks/today/top-losers/', 'body': 'Top Losers Today (US Stock Market) Below we list the top losers of the day sorted by largest price percentage decrease for the US stock market. This includes stocks from the NYSE, Nasdaq and AMEX stock exchanges. Rankings include all day volume including pre and post market hours (4:00am to 8:00pm in New York, NY, USA).'}, {'title': 'Top Daily Losses: US stocks posting the largest losses today - Yahoo ...', 'href': 'https://finance.yahoo.com/markets/stocks/losers/', 'body': "Yahoo Finance's list of top losing stocks, includes share price changes, trading volume, intraday highs and lows, and day charts for stocks posting the largest losses today"}, {'title': "Today's Top Stock Losers - Stock Analysis", 'href': 'https://stockanal

[{'title': 'Top Losers Today - Largest Stock Losses in the US Stock Market - Finsuba',
  'href': 'https://www.finsuba.com/leaderboards/us-stocks/today/top-losers/',
  'body': 'Top Losers Today (US Stock Market) Below we list the top losers of the day sorted by largest price percentage decrease for the US stock market. This includes stocks from the NYSE, Nasdaq and AMEX stock exchanges. Rankings include all day volume including pre and post market hours (4:00am to 8:00pm in New York, NY, USA).'},
 {'title': 'Top Daily Losses: US stocks posting the largest losses today - Yahoo ...',
  'href': 'https://finance.yahoo.com/markets/stocks/losers/',
  'body': "Yahoo Finance's list of top losing stocks, includes share price changes, trading volume, intraday highs and lows, and day charts for stocks posting the largest losses today"},
 {'title': "Today's Top Stock Losers - Stock Analysis",
  'href': 'https://stockanalysis.com/markets/losers/',
  'body': 'A list of the stocks with the highest per

In [4]:
from agentic_search.functions.web import get_webpages_soups_text_async

res = await get_webpages_soups_text_async([x["href"] for x in links])
res

['Top Losers Today - Largest Stock Losses in the US Stock Market Clear News News Cryptocurrencies Stocks Crypto Crypto News Leaderboards Rankings Stocks Stocks News Leaderboards Rankings Top Losers Today (US Stock Market) Below we list the top losers of the day sorted by largest price percentage decrease for the US stock market. This includes stocks from the NYSE, Nasdaq and AMEX stock exchanges. Rankings include all day volume including pre and post market hours (4:00am to 8:00pm in New York, NY, USA). Most Active Top Gainers Top Losers All Stocks Stocks Under $1 Stocks Under $2 Stocks Under $5 Stocks Under $10 Nasdaq Stocks NYSE Stocks NYSE American Stocks Premarket: Premarket Most Active Premarket Gainers Premarket Losers Regular Trading Hours: Trading Hours Most Active Trading Hours Gainers Trading Hours Losers After Hours: After Hours Most Active After Hours Gainers After Hours Losers Timeframe: Today Yesterday This Week Last Week This Month Last Month This Quarter Last Quarter Th

## test chains

In [5]:
from agentic_search.chains.web import get_web_search_queries_chain

chain = get_web_search_queries_chain()
res = chain.invoke({"query": user_query})
res


{'queries': ['latest top loser in the American stock market December 2024',
  'biggest stock market losers USA December 2024',
  'worst performing stocks in the US market December 2024',
  'largest stock declines in the American market December 2024',
  'top losing stocks on NYSE and NASDAQ December 2024',
  'recent stock market losers USA December 2024',
  'biggest stock drops in the US market December 2024']}

In [6]:
from agentic_search.chains.web import get_web_search_query_chain

chain = get_web_search_query_chain()
res = chain.invoke({"query": user_query})
res

{'query': 'latest top loser American stock market December 2024'}

## test tools

In [7]:
from agentic_search.tools.web import get_agentic_thorough_web_search_results_tool

res = await get_agentic_thorough_web_search_results_tool(user_query, 2)
# write results to file for consultation
with open("scratchpad.md", "w") as f:
    f.write(res)

[DEBUG] 
invoking thorough web search tool with query: what is the latest top loser in the American stock market

[DEBUG] 
serp results for query latest top loser in the American stock market December 2024: [{'title': 'Top Stock Losers In The Past Week - Stock Analysis', 'href': 'https://stockanalysis.com/markets/losers/week/', 'body': 'Top Gainers; Top Losers; Most Active; Premarket; After Hours; Market Newsletter. Stock Analysis Pro. Watchlist. ... 5 Years; Stock Indexes - 1 Week. S&P500-3.35%. Nasdaq 100-3.07%. Dow Jones-3.70%. Russell 2000-6.18%. Week Losers. Updated Dec 19, 2024. 20 Rows. Download. Indicators. No. Symbol Company Name Change 1W Stock Price Volume Market ...'}, {'title': 'Top Losing Stocks Today | Biggest Market Declines - Public.com', 'href': 'https://public.com/markets/top-losers', 'body': "Top Losers. Last updated Dec 23, 2024 at 1:05 AM. Discover today's top-losing stocks with real-time data to help investors and traders stay informed on the latest market shifts

In [8]:
from agentic_search.tools.web import get_agentic_quick_web_search_results_tool

res = await get_agentic_quick_web_search_results_tool(user_query, 6)
# write results to file for consultation
with open("scratchpad.md", "w") as f:
    f.write(res)

[DEBUG] 
invoking quickweb search tool with query: what is the latest top loser in the American stock market

[DEBUG] 
serp results for query latest top loser American stock market December 2024: [{'title': 'Top Losing Stocks Today | Biggest Market Declines - Public.com', 'href': 'https://public.com/markets/top-losers', 'body': "Top Losers. Last updated Dec 23, 2024 at 1:05 AM. Discover today's top-losing stocks with real-time data to help investors and traders stay informed on the latest market shifts. This data includes the 50 US-listed stocks with the steepest losses, with key financial metrics like daily change, market cap, trading volume, and daily highs and ..."}, {'title': 'Top Stock Losers In The Past Week - Stock Analysis', 'href': 'https://stockanalysis.com/markets/losers/week/', 'body': 'Top Gainers; Top Losers; Most Active; Premarket; After Hours; Market Newsletter. Stock Analysis Pro. Watchlist. ... 5 Years; Stock Indexes - 1 Week. S&P500-3.35%. Nasdaq 100-3.07%. Dow Jones

## test capabilities

In [1]:
from agentic_search.capabilities.web import get_web_search_results

res = await get_web_search_results("what is the ticker symbol of the biggest yoghurt company in the world")
res
# write results to file for consultation
with open("scratchpad.md", "w") as f:
    f.write(res["results"])

[DEBUG] 
invoking web search agent with messages: [SystemMessage(content='You are a precise research assistant with web search capabilities. Your tasks:\n1. Provide accurate, current information\n2. Synthesize multi-source information concisely\n3. Include citations and maintain objectivity\n\nFocus on authoritative, verifiable sources only.\n\nSkip web search if you are completely certain of information.\nOtherwise, perform targeted searches as needed.\nToday is 2024-12-24.', additional_kwargs={}, response_metadata={}), HumanMessage(content='what is the ticker symbol of the biggest yoghurt company in the worldAnswer in JSON format: {"content": "your results"}', additional_kwargs={}, response_metadata={}, id='449627b9-c753-4620-80b1-3fa4f71d37d5')]

[DEBUG] 
invoking quickweb search tool with query: biggest yogurt company in the world 2024

[DEBUG] 
serp results for query largest yogurt company in the world 2024: [{'title': '15 Largest Yogurt Companies And Brands In The World - Zippia'

In [2]:
input_headlines = [
    "UK gambling with climate targets over carbon capture, say campaigners - UK gambling with climate targets over carbon capture, say campaigners\n\nPotential IPCC rule changes could award planned carbon savings from burning US wood pellets to the exporter, not the importer\n\nThe UK government is gambling with its own climate targets on claims that the Drax power plant will create \u201cnegative emissions\u201d because new rules could hand the carbon savings to the US, campaigners say.The owners of the North Yorkshire power plant have promised ministers that a key project to capture the carbon emissions created from burning biomass wood pellets imported from US forests will count as negative emissions in Britain\u2019s carbon accounts.However, a working group convened by the UN\u2019s climate authority, the Intergovernmental Panel on Climate Change (IPCC), has begun meetings to draft rules for national greenhouse gas accounting which will apply to \u201ccarbon removal\u201d technologies from 2027 at the earliest.Campaigners at Biofuelwatch, a green group, have warned that there is \u201ca strong argument\u201d that the so-called negative emissions from bioenergy with carbon capture and storage, known as Beccs, should be \u201cattributed to the country where the wood comes from\u201d rather than where it is burned to generate electricity.Almuth Ernsting, a co-director at Biofuelwatch, said: \u201cWe cannot second-guess what the IPCC expert group will decide, but neither can the UK government.\u201dThe government is considering whether to extenda subsidy scheme that pays Drax about \u00a3500m a yearbeyond its 2027 deadline until the end of the decade. The FTSE 250 company, which agreed topay a \u00a325m fineearlier this year for misreporting its wood pellet sourcing, has already earned more than \u00a37bn in subsidies since work began to convert the former coal power plant to run on biomass in 2012.Ernsting said: \u201cIt is therefore entirely possible that, if the UK government were to grant billions of pounds in new subsidies to help Drax develop Beccs, this would benefit the greenhouse gas accounts of the USA, Canada and other countries exporting pellets to Drax \u2013 and not the UK.\u201dIn the past Drax has claimed its biomass generation is \u201ccarbon neutral\u201d because the emissions produced from its chimney stacks are offset by the emissions absorbed by the trees grown in North America to produce the pellets.It has quietly dropped the use of this wording since its own independent advisory board last year cautioned against repeating the claims. The company still insists that it will be a carbon-negative power plant once it fits carbon capture technology to its flues in the 2030s because it is a \u201cwidely accepted scientific view\u201d, according to a company spokesperson.This view has been disputed in recent years by a rising number of scientific studies from European academics. They fear the lag between when the emissions escape from power plant chimneys and when new trees are able to absorb carbon will create a \u201ccarbon debt\u201d which could accelerate the climate crisis in the near term.A Drax spokesperson said the company did not expect the IPCC expert group to amend the rules on Beccs projects because their work would focus on newer carbon capture technologies, such asdirect air capture. But in the minutes of the first meeting, seen by the Guardian, there was a presentation on the feasibility of developing new or updated carbon accounting methods for technologies including Beccs.The Department for Energy Security and Net Zero said: \u201cWe follow the agreed international approach for estimating and reporting greenhouse gas emissions under the UN Framework Convention on Climate Change and the Paris Agreement, and we expect the country that captures CO2 to continue to benefit from negative emissions.\u201cThe subsidies for large-scale biomass generators will end in 2027 and we are reviewing evidence on potential support beyond this,\u201d the spokesperson added.",
    "Shrinking waistlines and growing profits: the weight-loss drug boom  - Shrinking waistlines and growing profits: the weight-loss drug boom \n\nDrugs such as Wegovy amd Mounjaro are an attractive prospect for online businesses and traditional pharmacies as well as the firms that make them\n\nIt is a trend rooted in profit-making. Adverts featuring prescription-only weight-loss medications are splashed across the internet \u2013 and it is causing concern among experts.But the question remains: who is driving the boom?Consider drugs for other conditions, be it asthma or high blood pressure: a quick internet search might through up a couple of online pharmacies, but not page after page of results offering discount codes for consultations, special offers for returning customers or money-off emails.Yet when it comes to weight-loss jabs, it is a different story.Boots offers returning customers 10% off each time they reorder a weight-loss treatment \u2013 a promotion that appears at the bottom of a page featuring Wegovy, Mounjaro and other prescription-only medications.The online firm Pharmacy2U lists the prices of medications including Wegovy and Mounjaro together with a promotional code entitling customers to \u00a340 off their first four weeks\u2019 supply.Taking a different approach, Daily Chemist is emailing \u00a325 credits for weight-loss orders to customers, with a link that takes them directly to prescription-only weight loss medications. Such emails have been sent to people who have previously informed the pharmacy they have a healthy weight.A Boots spokesperson said promotional activity for Boots Online Doctor services was also compliant with applicable legislation, MHRA (Medicines and Healthcare products Regulatory Agency) guidance and ASA advertising codes.\u201cAll Boots Online Doctor prices are for the full service \u2013 from the online consultation to aftercare and support \u2013 and not for prescription medicines alone,\u201d they said.\u201cWeight-loss treatment is most effective when used consistently, alongside important lifestyle changes. We offer 10% off the Boots Online Doctor weight-loss service for returning patients to encourage them to stay with us to receive continuity of care.\u201dA spokesperson for Pharmacy2U said: \u201cAny discounts offered as part of our private online doctor service adhere to the guidelines set out by GPhC [General Pharmaceutical Council]and MHRA and are solely in relation to the provision of the service by our highly qualified clinicians. Patients have the ability to ask our GPs for a treatment recommendation as part of the consultation.\u201dDaily Chemist did not respond to a request for comment. The MHRA and ASA said they would assess the adverts to decide if they merited investigation.In each of these cases, the cited webpage was just one click away from the homepage of the online pharmacy, apparently contradicting advice set out by the Advertising Standards Agency (ASA).\u201cMarketers should ensure that the casually browsing consumer does not come across information relating POMs with ease,\u201dthe ASAsays in its advice on \u201cPrescription-only Medicines (websites)\u201d.Dr Piotr Ozieranski, from the University of Bath, who researches the political sociology of medicine and pharmaceuticals, said concerns were not confined to public-facing marketing, with Novo Nordisk \u2013 the pharmaceutical company behind Saxenda, Ozempic and Wegovy \u2013 having previously been in trouble for its practices.Indeed Novo Noridskwas suspended from theAssociation of the British Pharmaceutical Industry in 2023 for actions including sponsoring biased training to healthcare professionals around Saxenda, and offering individual health professionals apatient group direction (PGD)that would allow them to give the drug without a prescription from a doctor or other prescriber \u2013 a move that was deemed an inducement to prescribe.However, insiders in the pharmaceutical industry have stressed adverts featuring prescription-only weight-loss medications are not being driven by the pharmaceutical industry noting, the demand is already sky-high.Instead, it seems, it is an approach taken by the pharmacies themselves.The financial gains for pharmacies offering the weight-loss jabs are phenomenal: a month\u2019s supply costs a patient about \u00a3150 for the lowest dose, rising to about \u00a3250 a month for the highest.With about 60% of the UK\u2019s population living with obesity or overweight, and the jabs requiring ongoing use for weight loss to be maintained, industry insiders told the Guardian the medications could make pharmacies hundreds of thousands of pounds a year.Ozieranski notesthe company Pharmadoctor,which sells clinical service packages to pharmacists, has an onlinecalculator that pharmacists can use to estimatethe profits they could make.The upshot is that weight-loss medications are an attractive prospect for online businesses and bricks-and-mortarpharmacies that are already struggling to remain open.However, Ozieranski says it is the pharmaceutical companies that have created a parallel private market for such medications, while Prof Simon Capewell of the University of Liverpool, noted pharmaceutical companies gained financially from advertisements of the drugs.\u201cAnything that\u2019s going to promote visibility, promote acceptability, promote normality of this [type of] drug has got to be positive for the pharmaceutical companies,\u201d Capewell said.",
    "\u2018Bad things can happen\u2019: how will the world adjust to the Trump presidency? - \u2018Bad things can happen\u2019: how will the world adjust to the Trump presidency?\n\nBefore he has taken a single executive decision, countries around the world are positioning themselves for his impact\n\n\u201cA revisionist state has arrived on the scene to contest the liberal international order, and it is not Russia or China, it is the United States. It is Trump in the Oval Office, the beating heart of the free world. The incoming administration contests every element of the liberal international order \u2013 trade, alliances, migration, multilateralism, democratic solidarity and human rights.\u201cThe narrative now at home and abroad is that the US is not what we thought it was. Trump was not an aberration, not a bug, but a feature of American politics and of America\u2019s story.\u201dThis stark assessment of the impact of Donald Trump\u2019s return to the White House in January, by the Princeton University professor of politics John Ikenberry, leads him to a question: \u201cWill the new global order be determined less by the US and more by its legacy partners? Will they seek an alternative framework globally and regionally, or will they make bets to ride this out, and do the transactional politics that Trump is going to request?\u201dAlready, from Ankara to Brussels to Tehran and Moscow, the whole world seems catalysed and in motion as countries seek answers to versions of that question. Without Trump taking a single executive decision, they are positioning, responding and adjusting to the long shadow he represents. Even Trump himself seems a little unnerved at what his return is unleashing. \u201cThe world seems to be going a little crazy now,\u201d he recently admitted in Paris.Amid the craziness, three distinct forms of response to Trump are starting to emerge.An \u201cideologically aligned\u201d group is emboldened, including populists in Europe, Latin America and Israel who believe their often Russia-friendly brand of nationalism will benefit from being in the slipstream of America First. The breakup of the European Union, an Argentina-style chainsaw taken to regulation, a new security architecture with Russia, regime change in Tehran: all become possibilities.A second group, led by China, foresees a diplomatic shake-up in which America becomes an agent of instability, leading to some kind of globalised realignment. For Beijing \u2013 facing the threat of 50% tariffs \u2013 the silver lining is that Trump\u2019s willingness to treat friends as foes may create a leadership vacuum that China, as the so-called advocate of \u201cthe global majority\u201d, can exploit.It is one version of the \u201calternative framework\u201d of which Ikenberry speaks. Trump seems aware of that risk and is already threatening to impose 100% tariffs on the Brics countries \u2013 Brazil, Russia, India, China, South Africa, Iran, Egypt, Ethiopia, and the United Arab Emirates \u2013 if they try to replace the US dollar with another currency.The third, more phlegmatic, group is made up of America\u2019s \u201clegacy partners\u201d in Europe and the G7 group of liberal democracies. They still hope that with the right mix of argument, flattery and self-abnegation they can make a rational case that appeals to Trump\u2019s self-interest.Leaders of these nominal allies, however much they revile Trump\u2019s methods, look at American power and feel they have no choice but to interact with him. \u201cWe have to dance with whoever is on the dancefloor,\u201d Mark Rutte, the new Nato chief, said in February. The battle then becomes one for Trump\u2019s brain, and to persuade him that America\u2019s interests do not stop at its borders. But if this strategy does not work, insurance is being taken out by strengthening alternative frameworks. Polling published this month by the European Council on Foreign Relations, for instance, showed that right across Europe voters prefer more Europe to more Trump.That need not be surprising. Trump, it should be recalled, describes the EU as a \u201cnot-so-mini-China\u201d. His threat to impose tariffs has already stimulated an internal debate in Europe on how to respond. The president of the European Central Bank, Christine Lagarde, has been accused of running up the white flag by saying Europe should try to ward off tariffs by buying US arms and liquefied natural gas.The former WTO director general Pascal Lamy described her strategy to \u201ccalm this barking dog\u201d as \u201cabsolutely wrong\u201d. He said: \u201cYou never negotiate with the US, whatever the president, from a position of weakness. We must be strong by showing the importance of our market to US exporters.\u201dBut even he admits this will be testing since different countries have different levels of dependence on the US. \u201cOur strength is the size of our market and our unity. Our weakness is our lack of geostrategic consistency and our disunity.\u201dUkraineTrump\u2019s true existential threat to Europe lies in Ukraine. Without having revealed a detailed peace plan, Trump seems to want to threaten both sides \u2013 Vladimir Putin with rearmament in Ukraine, Volodymyr Zelenskyy with withdrawal of support \u2013 to force an end to the war.Asked recently if he was concerned about Trump\u2019s intentions, the outgoing head of the EU\u2019s foreign service, Josep Borrell, said: \u201cHow could I not be worried? Certainly I am. This is the big elephant in the room; this is the question. What will Europeans do if the new American administration is no longer supporting Ukraine? This is the question the Ukrainians were asking me when I was visiting Kyiv. It came from the last soldier to the president of Ukraine. I don\u2019t think anyone knows the answer.\u201dTrump complained: \u201cWe are in for $350bn. Europe is in for $150bn. That needs to be equalised. The war with Russia is important for everyone, but it\u2019s more important for Europe than us. We have a little thing called an ocean between us.\u201dFaced by the risk Trump will agree a deal with Putin over his head, Zelenskyy at the end of November took pre-emptive action. He proposed a ceasefire in which Ukraine would solely rely on diplomacy \u2013 not armed conflict \u2013 to regain the territory lost to Russia in the east since 2014, but in return the remaining part of Ukraine would be offered Nato membership, and not just the \u201cwell-lit bridge\u201d to membership at some point in the future.But Rutte has already rejected his plan, arguing that none of the key players, including Russia or the US, would accept Ukrainian membership of Nato now. Nato membership includes the key commitment that an attack on one member is an attack on all.That leaves Europe facing some fateful decisions and not much time in which to take them.Norbert R\u00f6ttgen, the German CDU foreign affairs specialist, said: \u201cEurope squandered the time it should have spent investing more heavily into the relationship [with the US] \u2013 including by building up its own defences \u2026 European leaders cannot simply shift the blame for their predicament to Washington.\u201dBorrell insists he did try to make Europe learn to speak \u201cthe language of power\u201d, and so be capable of acting more independently from the US. This required not just an increase in European defence spending, but a change in mindset that he admits has only just started. He recalls when he was asked how Europe would respond to the Russian attack on Ukraine in 2022, he instructed his officials to work up an option, and they came up with a plan to spend \u20ac50m (\u00a341m). \u201cI said \u201cAre you crazy? Put three 0s behind that. We are talking about a real war with bombs falling inside a European city.\u2019\u201dThe use of the European peace facility \u2013 an EU military aid fund \u2013 to help Ukraine, which was advocated by Borrell, was a significant step. Overall progress towards creating a European defence capability has been glacial, however. He blames a lack of political will, symbolised by the opposition within the EU to ending the national veto in all foreign policy decisions. Borrell explained: \u201cWe live in an ecosystem where you cannot speak, you cannot open your mouth if there is no unity. How many times did I have to say \u2018there is no European position on this\u2019? Everyone wants to keep their right to a veto, that is the reality.\u201dBorrell also found Europe\u2019s industrial defence base hollowed out. \u201cI have been begging for arms. It took three months just to ask for 1m rounds of ammunition. Russia is shooting 800,000 in a month.\u201d Since the Ukraine war started, he said, nearly 45% of all the equipment provided by EU nations came from outside the bloc.So a European plan to protect Ukraine is still needed, but Trump is having a galvanising impact.A statement issued earlier this month by foreign ministers from Germany, France, Poland, Italy and the UK after a meeting in Berlin contained a warning to Trump not to strike a deal with Putin behind Europe\u2019s back: \u201cThere can be no negotiations about peace in Ukraine without Ukrainians and without Europeans by their side.\u201dThe question is, given the shortfalls described by Borrell: can Europe realistically fill the gap if Trump ends US support? The plan is a European coalition of the willing, cooperating outside EU structures, providing security guarantees to Ukraine by stationing peacekeeping troops inside Ukraine along an as yet unnegotiated demarcation line. Unconfirmed leaks suggest the package includes as many as 40,000 troops, and a novel means to fund boosts to defence spending. The plan based on the model of postwar North and South Korea is so sensitive that no one yet wishes to discuss it in public.The key drivers are Poland\u2019s prime minister, Donald Tusk, in an alliance with the probable next German chancellor, Friedrich Merz, and the French president, Emmanuel Macron. The coalition needs to be credible enough both to convince Putin total victory is unachievable and to persuade Trump that Europe is reforming, and deserves American backing.The Middle East and ChinaBut just as Trump\u2019s victory has forced the principals in Europe to rethink their war, Middle Eastern leaders wonder how their region\u2019s proliferating and interconnected conflicts will be affected by Trump. Will he vacate the area, leaving the chaos to deepen, and allow other actors such as Turkey to regain their dominance and influence?Even Trump\u2019s closest allies seem divided on how he will react. Some US diplomats brief: \u201cTrump is not going to be like Bush or Obama. Sure, he\u2019ll take calls from Gulf leaders, but he is going to respond: \u2018That\u2019s your problem.\u2019\u201dOthers such as Mike Waltz, the president-elect\u2019s choice for national security adviser, say Trump is ready to make \u201cbig transformative deals\u201d. Waltz is already claiming credit for the Lebanon ceasefire agreed before the inauguration. \u201cEveryone\u2019s coming to the table because of President Trump,\u201d he claimed.Yet Trump\u2019s previous Middle East envoy Jason Greenblatt at the recent Doha forum for political dialogue said Trump saw no need to solve the Palestinian issue. \u201cMany of you think the Palestinian issue is the be-all and end-all. Not true. What the Trump administration did, and I think they will try to do again, is to decouple the Palestinian issue from everything else.\u201d He said that after the 7 October 2023 Hamas attacks on Israel \u201cthe Palestinian issue is so complicated and deeply entrenched, it is going to be hard to fix now\u201d.But that means the biggest deal \u2013 the one between Saudi Arabia and Israel \u2013 looks impossible for the moment. The Saudi crown prince, Mohammed bin Salman, has described what Israel has done in Gaza as a \u201cgenocide\u201d, and his diplomats have said normalisation with Israel and an expansion of the Abraham accords \u2013 bilateral agreements signed by Israel, the UAE and Bahrain during Trump\u2019s first term \u2013 is \u201coff the table without a clear pathway to a Palestinian state\u201d.Moreover, just like Europe, the Gulf states are looking to reduce their reliance on the US.Fawaz Gerges, a professor of the Middle East at the London School of Economics, pointed out: \u201cSaudi Arabia, one of the most important Middle Eastern powers dependent on the US, is positioning itself for the Trump administration by diversifying its foreign policy, deepening its relations with China and normalising with Iran.\u201dHe argued: \u201cThe Biden administration has insulted and humiliated its own Arab allies, all of which have been begging him to use leverage with the Israeli prime minister.\u201dThe US, he contended, by \u201cturning Israel into a military fortress, has prevented Israel from coming to terms with its neighbours. The US has not been a true friend of Israel since it has not sought to answer the question of: how do you anchor and integrate Israel into the social fabric in the region?\u201dA self-confident Israel sees no need to answer such questions. The \u201cmilitary fortress\u201d, after its display of dominance in 2024, regards 2025 as the year to complete the remaking of the Middle East that it has begun in Gaza, Lebanon and Syria.Within a week of the US election, the extremist Israeli cabinet member Bezalel Smotrich proclaimed that Trump\u2019s return meant that \u201c2025 will, with God\u2019s help, be the year of [Israeli] sovereignty in Judea and Samaria\u201d \u2013 the Israeli name for the West Bank. Annexation, an issue hotly contested within the first Trump administration, is back on the agenda, a prospect that terrifies the Gulf states.More importantly, the stars may finally have aligned for Netanyahu to attack Iran\u2019s nuclear sites, his objective for more than a decade. Anti-Tehran hawks will run Washington, Iranian air defences are weakened due to Israel\u2019s previous attack in 26 October, the \u201caxis of resistance\u201d is in even worse disrepair, Syria no longer exists as a military force, and Netanyahu claims after three conversations since the election he and Trump \u201csee eye to eye on the Iranian threat in all its components, and the danger posed by it\u201d.But Trump may not give Netanyahu the blank cheque he wants. He has sold himself to the American people as a leader that carries a big stick, but does not start wars. Iran\u2019s leadership, despite the internal divisions, has also been sending signals to prove that it wants to get back to the negotiating table with the US and have sanctions lifted.On the campaign trail Trump revealed that he had been prepared to make a deal with Iran \u201cwithin one week after the election\u201d if he had won in 2020. \u201cIt would have been a great deal for them. The only thing they cannot have is nuclear weapons.\u201dTrump may also not behave as many predict with China, focusing on the trade surplus, as opposed to an existential ideological war with the Chinese Communist party that does not animate him. China\u2019s president, Xi Jinping, has already been manoeuvring in Europe, in the global south and with business opinion to portray himself as a free trading stable partner committed to green energy.The contrast between the idiosyncratic decision-making in the White House and the predictability of China\u2019s modus operandi is also being promoted globally. This narrative may conceal more than it reveals, but it finds a ready reception in bodies such as the UN.It is also in stark contrast to the bleak description of Trump\u2019s vision set out by the incoming vice-president, JD Vance: \u201cPeople are genuinely afraid that if they don\u2019t listen to him, bad things can happen, and there are consequences for disregarding him.\u201d",
    "Labour urged to lead global debt relief effort as cost of repayments soars - Labour urged to lead global debt relief effort as cost of repayments soars\n\nCampaigners call for action in 2025, as crippling repayments prevent spending to cut poverty and tackle climate crisis\n\nCampaigners are urging Labour to lead the charge for global debt relief in 2025, as new analysis shows lower-income countries spent 15% of their revenue on repayments this year \u2013 the highest level for three decades.Calculations by the charity Debt Justice, based on data from the World Bank, show repayments from poorer countries bottomed out at 4.4% of income in 2011 but have since trebled.\u201cHigh debt payments are preventing the public spending needed to cut poverty and tackle the climate emergency,\u201d said Debt Justice\u2019s policy director Tim Jones.Gideon Rabinowitz, director of policy and advocacy at Bond, the anti-poverty coalition, said: \u201cLower-income countries are being crushed by unsustainable debt, therefore debt cancellation must be on the international agenda next year. This debt crisis is a symptom of a broken global financial system which reproduces this recurring cycle of debt crises.\u201dInteractiveAmong the countries facing the highest repayments as a share of income in 2024 were Angola (65%), Laos (52%), Pakistan (43%) and Egypt (43%).The International Monetary Fund (IMF) recentlyagreed a fresh $7bn loan to Pakistan\u2013 the latest of a string of bailouts \u2013 despite warning that the country faced \u201chigh risks and a narrow path\u201d to debt sustainability.Angola struck a deal earlier this year with the state-backed China Development Bank \u2013 its largest creditor \u2013 to avert a debt crunch. Chinese lenders, state and private, account for 13% of low-income countries\u2019 repayments.Writing in the World Bank\u2019s recentInternational Debt Report, its chief economist, Indermit Gill, warned: \u201cit\u2019s time to face the reality: the poorest countries facing debt distress need debt relief if they are to have a shot at lasting prosperity.\u201dLabour\u2019s manifesto at the general election cited \u201ctackling unsustainable debt\u201d as a priority for its global development agenda.With many global debt contracts governed by UK law, Debt Justice is also calling on the Labour government to pass legislation to ensure private sector bondholders are forced to take a loss, as part of future debt relief deals. Development minister Anneliese Dodds backed such a law in opposition.\u201cThe government needs to turn words into action in 2025 by pushing for an effective debt cancellation scheme and passing a law to ensure private lenders take part in debt relief,\u201d Jones said.Debt outlays for lower-income countries are expected to remain high in 2025, at 14%.With some of it denominated in dollars, repayments would increase further in countries\u2019 domestic currencies, if the greenback surges on foreign exchange markets as a result of Donald Trump\u2019s policies as many analysts predict.Pope Francis has called for debt relief as part of a jubilee year for Catholics, in 2025 \u2013 the first since 2000. He said this month: \u201cIn the spirit of this jubilee year, I urge the international community to work towards forgiving foreign debt in recognition of the ecological debt existing between the North and the South of this world. This is an appeal for solidarity, but above all for justice.\u201dThe crippling financial burden for developing countries in the late 1990s prompted the successful Jubilee 2000 campaign for debt forgiveness, which ultimately resulted in $130bn (\u00a3102bn) of debt being written off. As chancellor, Gordon Brown took a leading role in that campaign.Since coming to power in July, Keir Starmer\u2019s government has opted not to reverse the Tories\u2019 decision to dismantle the Department for International Development (DfID), which was set up by Labour in 1997, but abolished by Boris Johnson.Nor has Labour said when it will reverse cuts to overseas aid made by Johnson\u2019s government, whichditched the pledge of spending 0.7% of national income on development. Rachel Reeves did not mention overseas aid or development in her budget speech in October.A UK government spokesperson said: \u201cWe are highly concerned about the debt challenges facing low-income countries. Supporting developing countries to tackle unsustainable debt is a key development priority of this government.\u201cThe UK is playing its full part alongside international partners to support countries who face debt distress, through international fora, as well as looking to strengthen the system for the future.\u201dThe World Bank data covers 84 countries categorised as low income, or lower-middle income, as well as upper middle-income countries classified as small island developing states.",
]

from agentic_search.capabilities.text import get_running_summary


In [3]:
res_openai = get_running_summary(input_headlines, "openai")
res_openai

{'summary': "- **UK Climate Targets and Carbon Capture**: The UK may miss climate targets due to potential IPCC rule changes that could allocate carbon savings from burning US wood pellets to the exporter. The Drax power plant claims negative emissions, but there is debate over whether emissions should be attributed to the origin country. The UK government might extend subsidies for Drax, benefiting exporting countries' greenhouse gas accounts. Drax has been criticized and fined for misreporting wood pellet sourcing.\n\n- **Weight-Loss Drug Boom**: The market for weight-loss drugs like Wegovy and Mounjaro is expanding, driven by online businesses and pharmacies. These drugs are heavily advertised online, raising concerns among experts. Pharmacies offer discounts to attract customers, despite regulatory guidelines. A month's supply costs between £150 and £250, with significant financial gains for pharmacies. Pharmaceutical companies benefit from a parallel private market.\n\n- **Global 

In [4]:
res = get_running_summary(input_headlines)
res

{'summary': "The UK government is considering extending a subsidy scheme for Drax's biomass power plant, which could benefit the US and other countries exporting wood pellets. Campaigners warn that this would attribute negative emissions to the wrong country.\n\nDonald Trump's presidency has caused concern globally, as his administration contests every element of the liberal international order – trade, alliances, migration, multilateralism, democratic solidarity, and human rights.\n\nThree distinct forms of response to Trump are emerging: an 'ideologically aligned' group emboldened by populists in Europe, Latin America, and Israel; a second group led by China, foreseeing a diplomatic shake-up where America becomes an agent of instability; and a third, more phlegmatic group made up of America's 'legacy partners' in Europe and the G7 group of liberal democracies.\n\nEurope faces fateful decisions regarding Ukraine, with Trump threatening to withdraw support. A European coalition of the 